# Time Series Analysis with SARIMAX

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

In [10]:
df = pd.read_csv("nyc_data.csv", index_col=0, parse_dates=True)
df = df.rename(columns={"Demand":"y"})
df.head(1)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305


In [11]:
X = df.iloc[:, 1:]
X.head(2)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,
2015-01-01,0,0,0,3.68,41.305
2015-01-02,0,0,0,4.73,131.574


### Stationarity

In [12]:
pvalue = adfuller(x=df.y)[1]
if pvalue < 0.05:
    print(f"The Time Series is stationary. The p-value is {pvalue:.4f}")
else:
    print(f"The Time Series is not stationary. The p-value is {pvalue:.4f}")

The Time Series is not stationary. The p-value is 0.3768


In [13]:
pvalue = adfuller(x=df.y.diff().dropna())[1]
if pvalue < 0.05:
    print(f"The Time Series is stationary. The p-value is {pvalue:.4f}")
else:
    print(f"The Time Series is not stationary. The p-value is {pvalue:.4f}")

The Time Series is stationary. The p-value is 0.0000
